In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
import torch
from torch import nn
import torch.nn.functional as F

import syft as sy
from syft import make_plan
from syft.lib.python.string import String
from syft import logger
from torch.utils.data import Dataset, DataLoader


In [4]:
# train_set = datasets.load_dataset("ptb_text_only", split="train")

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'distilroberta-base'

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)

alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()
t = String("abcd")
tp = t.send(alice_client)

x_ptr = String("a simple sequence").send(alice_client)
x = "a simple sequence"
out = x_ptr.strip().split()

result = tokenizer._tokenizer.encode(x_ptr)


TypeError: TextInputSequence must be str

In [9]:
dir(tokenizer._tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'decoder',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'from_buffer',
 'from_file',
 'from_str',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'model',
 'no_padding',
 'no_truncation',
 'normalizer',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'post_processor',
 'pre_tokenizer',
 'save',
 'to_str',
 'token_to_id',
 'train',
 'train_from_iterator',
 'truncation']

In [21]:
# Third party
from google.protobuf import json_format
from tokenizers import Tokenizer


def object2proto(object: Tokenizer):
    tokenizer_json_str = object.to_str()
    tokenizer_proto = json_format.Parse(tokenizer_json_str)
    
object2proto(tokenizer._tokenizer)

TypeError: Parse() missing 1 required positional argument: 'message'

In [5]:
# model.classifier

class DummyDataset(Dataset):
    def __init__(self):
        self.data = torch.LongTensor(100, 20).random_(0, 10000)
        self.labels = torch.LongTensor(100).zero_()
        
    def __getitem__(self, i):
        return self.data[i]
    
    def __len__(self):
        return self.data.size(0)
    
class RNN(nn.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.emb = nn.Embedding(10000, 100)
#         self.rnn = nn.LSTM(input_size=100, hidden_size=100, num_layers=100, batch_first=True)
        
    def forward(self, x):
        emb = self.emb(x)
        return emb.mean(-1)
#         return self.rnn(emb)[0][:, 0]

model = RNN()
x = DummyDataset()[0]
model(x)

tensor([ 0.1147,  0.0831, -0.1408,  0.0703, -0.0130, -0.0653,  0.1901,  0.0604,
         0.0057,  0.0365, -0.0783,  0.0224,  0.0190, -0.0809, -0.0380, -0.0074,
         0.0478, -0.0206, -0.0096,  0.0630], grad_fn=<MeanBackward1>)

In [12]:
class SyRNN(sy.SyModule):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.emb = nn.Embedding(10000, 100)
#         self.rnn = nn.LSTM(input_size=100, hidden_size=100, num_layers=100, batch_first=True)
        
    def forward(self, x):
        emb = self.emb(x)
        return emb.mean(-1)
#         return self.rnn(emb)[0][:, 0]
    
rnn = SyRNN(input_size=(10, 20), input_dtype=torch.long)
train_loader = DataLoader(DummyDataset(), batch_size=10, shuffle=True)
dummy_dl = sy.lib.python.List([next(iter(train_loader))])
remote_torch = alice_client.torch

In [17]:
@make_plan
def train(dl = dummy_dl, model=rnn):
    x = next(iter(dl))[0]
    return model(x=x)

RECOMPILING
RECOMPILING


In [58]:
@make_plan
def test_forward(x=torch.LongTensor(5, 10).random_(0, 10), mask=torch.ones([5, 10], dtype=torch.long), model=model):
    return model(x, mask).logits

AttributeError: 'RobertaForSequenceClassification' object has no attribute '_forward_plan'

In [57]:
test_forward(model=model)

tensor([[0.2123, 0.1276],
        [0.2219, 0.1262],
        [0.2247, 0.1128],
        [0.2033, 0.1121],
        [0.2263, 0.1218]], grad_fn=<AddmmBackward>)

In [74]:
torch.randn(1, dtype="long")

TypeError: randn() received an invalid combination of arguments - got (int, dtype=str), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [5]:
torch.rand(10, dtype=torch.long)

RuntimeError: "check_uniform_bounds" not implemented for 'Long'

In [9]:
torch.rand(10).type(torch.long)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
tokenizer

PreTrainedTokenizer(name_or_path='distilroberta-base', vocab_size=50265, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [29]:


x = String("abcd")
x_ptr = x.send(alice_client)

In [37]:
def test_tokenizer(x=String("abcd"), tokenizer=tokenizer):
    return tokenizer(x)

In [39]:
test_tokenizer(x=String("abcd"))

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).